In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
import lidar_sim_class_declarations
import importlib
from datetime import datetime
import pandas as pd


importlib.reload(lidar_sim_class_declarations)

output_path = 'C:/Users/allen/OneDrive/Desktop/Work/Scripts/Lidar Simulation/output_008/'

# Leaf Density ratio
# redwood_leaf_density_ratio = 2500  # Leaves per m3
# guiana_leaf_density_ratio = 87.5  # Leaves per m3  Original 8750 number divided by 100 for more realistic density per m3
# guiana_leaf_density_ratio = 8750/20  # Leaves per m3  Original 8750 number divided by 20 for more realistic density per m3 = 437.5

radius = 33.3 / 2
section_range = 0.1  # specifies section size.  (0.1 means section is per 0.1 meters)

In [2]:
def get_date_subfolder():
    current_date = datetime.now()
    return current_date.strftime('%Y_%m_%d')

def load_objects_from_file(filename, subfolder=""):
    # Adjust the filename to include the subfolder
    if subfolder:
        filename = os.path.join(subfolder, filename)
        
    # Load the objects from file
    with open(filename, 'rb') as f:
        objects_list = pickle.load(f)
    
    return objects_list

In [3]:
redwood_tree_objects_branch_orig = load_objects_from_file("redwood_tree_objects_branch_orig.pkl", subfolder="object_data")
redwood_tree_objects_0_01_branch = load_objects_from_file("redwood_tree_objects_branch_origB10th.pkl", subfolder="object_data")
guiana_tree_objects_leaf_orig = load_objects_from_file("guiana_tree_objects_leaf_orig.pkl", subfolder="object_data")
guiana_tree_objects_leaf_div10 = load_objects_from_file("guiana_tree_objects_leaf_div10.pkl", subfolder="object_data")
guiana_tree_objects_leaf_div30 = load_objects_from_file("guiana_tree_objects_leaf_div30.pkl", subfolder="object_data")

redwood_circle_objects_orig = load_objects_from_file("redwood_circle_objects_orig.pkl", subfolder="object_data")
redwood_circle_objects_0_01_branch = load_objects_from_file("redwood_circle_objects_origB10th.pkl", subfolder="object_data")
guiana_circle_objects_leaf_orig = load_objects_from_file("guiana_circle_objects_leaf_orig.pkl", subfolder="object_data")
guiana_circle_objects_leaf_div10 = load_objects_from_file("guiana_circle_objects_leaf_div10.pkl", subfolder="object_data")
guiana_circle_objects_leaf_div30 = load_objects_from_file("guiana_circle_objects_leaf_div30.pkl", subfolder="object_data")

date_subfolder = get_date_subfolder()
csv_subfolder = "info_csv"
csv_output_path = os.path.join(output_path, date_subfolder, csv_subfolder)

print(csv_output_path)

C:/Users/allen/OneDrive/Desktop/Work/Scripts/Lidar Simulation/output_008/2024_10_16\info_csv


In [4]:
for circle in redwood_circle_objects_orig:
    print(f"Circle: {circle.circle_name}")
    # print(circle.acc_LAI_leaf)
    # print(circle.acc_LAI_branch)
    print(circle.waveform)
    print(circle.h1)

Circle: Circle(3, 1)
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.

In [5]:
def tree_info_to_df(circle_objects):
    df_list = []

    for circle in circle_objects:
        df = pd.DataFrame()

        tree_indices = circle.tree_indices
        tree_percentages = circle.crown_percentages
        tree_heights = circle.tree_heights
        crown_volumes = circle.crown_vol_in_circle
        leaf_areas = circle.adj_leaf_areas
        branch_areas = circle.adj_branch_areas
        total_veg_areas = circle.total_tree_canopy_veg_area

        df['tree_indices'] = tree_indices
        df['tree_percentages'] = tree_percentages
        df['tree_heights'] = tree_heights
        df['crown_volumes'] = crown_volumes
        df['leaf_areas'] = leaf_areas
        df['branch_areas'] = branch_areas
        df['total_veg_areas'] = total_veg_areas

        df_list.append(df)

    return df_list

def output_saveCSV(sitename, df_list, path, circle_names):
    # Step 1: Create a list to hold DataFrames with circle names as headers
    combined_dfs = []

    # Step 2: Loop through each DataFrame in df_list and combine with circle names as headers
    for i, df in enumerate(df_list):
        # Create a DataFrame with circle name as header
        header_df = pd.DataFrame({circle_names[i]: ['']})
        
        # Concatenate header_df and df along the rows axis
        combined_df = pd.concat([header_df, df], ignore_index=True)
        combined_dfs.append(combined_df)

    # Step 3: Concatenate all DataFrames in combined_dfs along the columns axis
    final_df = pd.concat(combined_dfs, axis=1)

    # Step 4: Create the directory path if it doesn't exist
    basic_tree_info_path = os.path.join(path, "basic_tree_info")
    if not os.path.exists(basic_tree_info_path):
        os.makedirs(basic_tree_info_path)

    # Step 5: Save the combined DataFrame to CSV
    final_df.to_csv(os.path.join(basic_tree_info_path, f'{sitename}_basic_tree_info.csv'), index=False)


In [6]:
redwood_basic_tree_df_list = tree_info_to_df(redwood_circle_objects_orig)
guiana_basic_tree_df_list = tree_info_to_df(guiana_circle_objects_leaf_orig)

circle_names = [circle.circle_name for circle in redwood_circle_objects_orig]

output_saveCSV("redwood", redwood_basic_tree_df_list, csv_output_path, circle_names)
output_saveCSV("guiana", guiana_basic_tree_df_list, csv_output_path, circle_names)

In [7]:
def output_GORT_plusLAI(circle_objects, path, site, type, scale_qualifier):
    circle_names = [circle.circle_name for circle in circle_objects]
    crown_volumes = [np.sum(circle.crown_vol_in_circle) for circle in circle_objects]
    pixel_area = np.array([circle.pixel_area for circle in circle_objects])
    tree_density = np.array([circle.tree_density for circle in circle_objects])
    h1 = np.array([circle.h1 for circle in circle_objects])
    h2 = np.array([circle.h2 for circle in circle_objects])
    mean_crown_d2 = np.array([circle.mean_crown_d2 for circle in circle_objects])
    mean_crown_d1 = np.array([circle.mean_crown_d1 for circle in circle_objects])

    if(type == "leaf"):
        circle_Fa = np.array([circle.new_Fa for circle in circle_objects])
    if(type == "veg"):
        circle_Fa = np.array([circle.Fa_b for circle in circle_objects])

    total_LAI = (crown_volumes*circle_Fa)/pixel_area

    total_LAI_df = pd.DataFrame()

    total_LAI_df['circle_names'] = circle_names
    total_LAI_df['tree_density'] = tree_density
    total_LAI_df['circle_Fa'] = circle_Fa
    total_LAI_df['h1'] = h1
    total_LAI_df['h2'] = h2
    total_LAI_df['mean_crown_d2'] = mean_crown_d2
    total_LAI_df['mean_crown_d1'] = mean_crown_d1
    total_LAI_df['total_LAI'] = total_LAI

    GORT_inputs_path = os.path.join(path, "GORT_inputs_tables")
    if not os.path.exists(GORT_inputs_path):
        os.makedirs(GORT_inputs_path)

    filename = f'{site}_{scale_qualifier}_{type}_GORT_inputs_andLAI.csv'

    total_LAI_df.to_csv(f'{GORT_inputs_path}/{filename}', index=False)
    print(f'wrote to: {GORT_inputs_path}/{filename}')

    return total_LAI_df

In [8]:
redwood_total_LAI_leaf_df_orig = output_GORT_plusLAI(redwood_circle_objects_orig, csv_output_path, "redwood", "leaf", "orig")
redwood_total_LAI_veg_df_orig = output_GORT_plusLAI(redwood_circle_objects_orig, csv_output_path, "redwood", "veg", "orig")
redwood_total_LAI_leaf_df_0_01_branch = output_GORT_plusLAI(redwood_circle_objects_0_01_branch, csv_output_path, "redwood", "leaf", "origB100th")
redwood_total_LAI_veg_df_0_01_branch = output_GORT_plusLAI(redwood_circle_objects_0_01_branch, csv_output_path, "redwood", "veg", "origB100th")

guiana_total_LAI_leaf_df_orig = output_GORT_plusLAI(guiana_circle_objects_leaf_orig, csv_output_path, "guiana", "leaf", "orig")
guiana_total_LAI_veg_df_orig = output_GORT_plusLAI(guiana_circle_objects_leaf_orig, csv_output_path , "guiana", "veg", "orig")
guiana_total_LAI_leaf_df_div10 = output_GORT_plusLAI(guiana_circle_objects_leaf_div10, csv_output_path, "guiana", "leaf", "LD10th")
guiana_total_LAI_veg_df_div10 = output_GORT_plusLAI(guiana_circle_objects_leaf_div10, csv_output_path, "guiana", "veg", "LD10th")
guiana_total_LAI_leaf_df_div30 = output_GORT_plusLAI(guiana_circle_objects_leaf_div30, csv_output_path, "guiana", "leaf", "origFD30th ")
guiana_total_LAI_veg_df_div30 = output_GORT_plusLAI(guiana_circle_objects_leaf_div30, csv_output_path, "guiana", "veg", "origFD30th")

wrote to: C:/Users/allen/OneDrive/Desktop/Work/Scripts/Lidar Simulation/output_008/2024_10_16\info_csv\GORT_inputs_tables/redwood_orig_leaf_GORT_inputs_andLAI.csv
wrote to: C:/Users/allen/OneDrive/Desktop/Work/Scripts/Lidar Simulation/output_008/2024_10_16\info_csv\GORT_inputs_tables/redwood_orig_veg_GORT_inputs_andLAI.csv
wrote to: C:/Users/allen/OneDrive/Desktop/Work/Scripts/Lidar Simulation/output_008/2024_10_16\info_csv\GORT_inputs_tables/redwood_origB100th_leaf_GORT_inputs_andLAI.csv
wrote to: C:/Users/allen/OneDrive/Desktop/Work/Scripts/Lidar Simulation/output_008/2024_10_16\info_csv\GORT_inputs_tables/redwood_origB100th_veg_GORT_inputs_andLAI.csv
wrote to: C:/Users/allen/OneDrive/Desktop/Work/Scripts/Lidar Simulation/output_008/2024_10_16\info_csv\GORT_inputs_tables/guiana_orig_leaf_GORT_inputs_andLAI.csv
wrote to: C:/Users/allen/OneDrive/Desktop/Work/Scripts/Lidar Simulation/output_008/2024_10_16\info_csv\GORT_inputs_tables/guiana_orig_veg_GORT_inputs_andLAI.csv
wrote to: C:/Us

In [9]:
circle_Fa = np.array([circle.new_Fa for circle in redwood_circle_objects_orig])
print(circle_Fa)
circle_Fa_b = np.array([circle.Fa_b for circle in redwood_circle_objects_0_01_branch])
print(circle_Fa_b)

[0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15]
[0.49471091 0.49322434 0.48672066 0.48889461 0.48264611 0.49084427
 0.49687946 0.48175771 0.48235298]
